In [82]:
import os
import re
import numpy as np
import scipy
import pandas as pd
from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import model_selection
import time

In [83]:
#Import file with features extracted in the CNN process
path = '/Users/chloe/Desktop/DataScientest/project/Picasso/pred_cnn.csv'
df=pd.read_csv(path)

In [84]:
#Dataframe preparation for machine learning
df['label'] = df['label'].str.replace('.jpg', '')
df['label'] = df['label'].str.replace('\d+', '')
df['label'] = df['label'].str.replace('ü','u')
df.drop(labels='Unnamed: 0.1',axis=1, inplace=True)
df.drop(labels='Unnamed: 0',axis=1, inplace=True)

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8355 entries, 0 to 8354
Columns: 129 entries, 0 to label
dtypes: float64(128), object(1)
memory usage: 8.2+ MB


In [86]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,label
0,0.0,12.782750,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,10.807129,0.0,1.452163,0.0,0.000000,0.000000,0.0,0.0,Albrecht Durer
1,0.0,12.510038,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.451544,...,0.0,15.300602,0.0,1.276377,0.0,0.687523,0.000000,0.0,0.0,Albrecht Durer
2,0.0,11.358869,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,17.039213,0.0,2.486779,0.0,0.000000,1.304097,0.0,0.0,Albrecht Durer
3,0.0,11.412414,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,19.503704,0.0,5.480277,0.0,0.000000,0.000000,0.0,0.0,Albrecht Durer
4,0.0,6.969485,0.0,0.954874,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,16.321180,0.0,5.468986,0.0,0.000000,0.000000,0.0,0.0,Albrecht Durer


In [87]:
#dataframe with features only
data = df.iloc[:,:128]
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.0,12.782750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,10.807129,0.0,1.452163,0.0,0.000000,0.0,0.0,0.0
1,0.0,12.510038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.451544,...,0.0,0.0,15.300602,0.0,1.276377,0.0,0.687523,0.0,0.0,0.0


In [88]:
#dataframe with label only
target = df['label']

In [89]:
#training set and testing set establishment
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

In [90]:
#default SVM classifier
clf= svm.SVC()

In [91]:
start = time.time()
clf.fit(X_train,y_train)
end = time.time()
print(end - start)

21.50014328956604


In [92]:
y_pred = clf.predict(X_test)
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Class réelle'], colnames = ['Classe prédite'])
confusion_matrix

Classe prédite,Albrecht Durer,Alfred Sisley,Amedeo Modigliani,Andrei Rublev,Andy Warhol,Camille Pissarro,Caravaggio,Claude Monet,Diego Rivera,Diego Velazquez,...,Pieter Bruegel,Raphael,Rembrandt,Rene Magritte,Salvador Dali,Sandro Botticelli,Titian,Vasiliy Kandinskiy,Vincent van,William Turner
Class réelle,,,,,,,,,,,,,,,,,,,,,
Albrecht Durer,60,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
Alfred Sisley,0,52,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,0
Amedeo Modigliani,0,0,25,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12,0
Andrei Rublev,0,0,0,15,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,8,0
Andy Warhol,0,0,0,0,27,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15,0
Camille Pissarro,0,0,0,0,0,14,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
Caravaggio,0,0,0,0,0,0,11,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Claude Monet,0,1,0,0,0,0,0,10,0,0,...,0,0,0,0,0,0,0,0,6,0
Diego Rivera,0,0,0,0,0,0,0,0,10,0,...,0,0,0,0,0,0,0,0,2,0


In [93]:
confusion_matrix.to_csv(r'/Users/chloe/Desktop/DataScientest/project/Picasso/confusion_matrix_SVM_0.csv')

In [94]:
clf.score(X_test,y_test)

0.7660083782166367

In [95]:
#search for the best hyperparameters
parametres={'C': [0.1,1,10],'kernel': ['rbf', 'linear','poly'],'gamma': [0.001, 0.1, 0.5]}

In [96]:
grid_clf=model_selection.GridSearchCV(estimator=clf,param_grid =parametres )

In [97]:
start = time.time()
grille=grid_clf.fit(X_train, y_train)
end = time.time()
print(end - start)

706.3154199123383


In [98]:
print(pd.DataFrame.from_dict(grille.cv_results_).loc[:,["params","mean_test_score"]]) 

                                            params  mean_test_score
0      {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}         0.830940
1   {'C': 0.1, 'gamma': 0.001, 'kernel': 'linear'}         0.887193
2     {'C': 0.1, 'gamma': 0.001, 'kernel': 'poly'}         0.689258
3        {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}         0.105925
4     {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}         0.887193
5       {'C': 0.1, 'gamma': 0.1, 'kernel': 'poly'}         0.866846
6        {'C': 0.1, 'gamma': 0.5, 'kernel': 'rbf'}         0.105925
7     {'C': 0.1, 'gamma': 0.5, 'kernel': 'linear'}         0.887193
8       {'C': 0.1, 'gamma': 0.5, 'kernel': 'poly'}         0.866846
9        {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}         0.898115
10    {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}         0.885996
11      {'C': 1, 'gamma': 0.001, 'kernel': 'poly'}         0.796080
12         {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}         0.105925
13      {'C': 1, 'gamma': 0.1, 'kernel': 'linear

/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py

In [99]:
#best parameters
grid_clf.best_params_

{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}

In [100]:
grid_clf.score(X_test,y_test)

0.9006582884500299